#### Import packages

In [5]:
import torch
import os
from accelerate import Accelerator, DataLoaderConfiguration
from denoising_diffusion_pytorch import Unet, GaussianDiffusion, Trainer

#### Initiate the model

In [6]:
accelerator = Accelerator(mixed_precision='fp16')
model = Unet(
    dim = 64,
    dim_mults = (1, 2, 4, 8)
).cuda()

diffusion = GaussianDiffusion(
    model,
    image_size = 256,
    timesteps = 1000,   # number of steps
   

).cuda()

trainer = Trainer(
    diffusion,
    '../data/neo_usable',
    train_batch_size = 32,
    train_lr = 2e-5,
    train_num_steps = 1000,         # total training steps
    gradient_accumulate_every = 2,    # gradient accumulation steps
    ema_decay = 0.995,                # exponential moving average decay
    amp = True                        # turn on mixed precision
)

trainer.train()

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001E084A20680>
Traceback (most recent call last):
  File "c:\Programing and shit\Python11\Lib\site-packages\torch\utils\data\dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "c:\Programing and shit\Python11\Lib\site-packages\torch\utils\data\dataloader.py", line 1436, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
                                   ^^^^^^^^^^^^^^^^^^^^
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'
  0%|          | 0/1000 [01:24<?, ?it/s]


KeyboardInterrupt: 

#### Save images

In [ ]:
save_dir = '../data/neo_diffusion'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
def generate_and_save_samples():
    for i in range(10):  # Adjust the number of samples you want to generate
        # Generate a sample image using the model
        generated_image = diffusion.sample(batch_size=1)

        # Convert the image to a format that can be saved (e.g., NumPy array)
        generated_image = generated_image.clamp(0.0, 1.0)  # ensure pixel values are within [0, 1]
        generated_image = (generated_image * 255).byte().cpu().numpy()

        # Save the image to ../data/neo_diffusion
        save_path = os.path.join(save_dir, f'generated_image_{i}.png')
        save_image(generated_image[0], save_path)

        print(f"Saved generated image {i} to {save_path}")

# Function to save generated image using PyTorch's save_image utility
from torchvision.utils import save_image

# Call the function to generate and save images
generate_and_save_samples()